# 通过subprocess获取数据库输出

In [1]:
from subprocess import Popen, PIPE

def mysqlPin(user,passwd, host):
    try:
        conn = 'mysql -u%s -p%s -h%s -B' % (user,
                                            passwd, host)
        # conn = 'mysql -u%s -p%s -h%s -s'%(self.user,self.passwd,self.host)
        Pin = Popen(conn, shell=True, stdin=PIPE, stdout=PIPE, stderr=PIPE)
        return Pin
    except Exception as e:
        print(e)
p=mysqlPin('root', 'paw', '127.0.0.1')
p.stdin.write('select * from sakila.film  limit 0,5;'.encode('ascii')+b'\n')
stdout=p.communicate()[0]
print(stdout.decode('utf-8'))

film_id	title	description	release_year	language_id	original_language_id	rental_duration	rental_rate	length	replacement_cost	rating	special_features	last_update
1	ACADEMY DINOSAUR	A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies	2006	1	NULL	6	0.99	86	20.99	PG	Deleted Scenes,Behind the Scenes	2006-02-15 05:03:42
2	ACE GOLDFINGER	A Astounding Epistle of a Database Administrator And a Explorer who must Find a Car in Ancient China	2006	1	NULL	3	4.99	48	12.99	G	Trailers,Deleted Scenes	2006-02-15 05:03:42
3	ADAPTATION HOLES	A Astounding Reflection of a Lumberjack And a Car who must Sink a Lumberjack in A Baloon Factory	2006	1	NULL	7	2.99	50	18.99	NC-17	Trailers,Deleted Scenes	2006-02-15 05:03:42
4	AFFAIR PREJUDICE	A Fanciful Documentary of a Frisbee And a Lumberjack who must Chase a Monkey in A Shark Tank	2006	1	NULL	5	2.99	117	26.99	G	Commentaries,Behind the Scenes	2006-02-15 05:03:42
5	AFRICAN EGG	A Fast-Paced Documentary of a Pastry Chef And 

returncode 返回管道语句执行后的结果，0表示成功

In [2]:
assert p.returncode == 0, 'Popen cmd failed.'

# communicate命令

与子进程进行交互。

可选参数input指定发送到子进程的参数。

Communicate()返回一个元组：(stdoutdata, stderrdata)。

注意：如果希望通过进程的stdin向其发送数据，在创建Popen对象的时候，参数stdin必须被设置为PIPE。同样，如果希望从stdout和stderr获取数据，必须将stdout和stderr设置为PIPE。

In [3]:
p=mysqlPin('root', 'paw', '127.0.0.1')
p.stdin.write('use sakila;'.encode('ascii')+b'\n')
stdout=p.communicate(input='show tables;'.encode('ascii'))[0]
print(stdout.decode('utf-8'))

Tables_in_sakila
actor
actor_info
address
category
city
country
customer
customer_list
film
film_actor
film_category
film_list
film_text
inventory
language
nicer_but_slower_film_list
payment
rental
sales_by_film_category
sales_by_store
staff
staff_list
store



# 连接多管道

类似 ls -l | grep mysql 即，在当前目录下，查询文件名称为mysql得到行

我们这里实现从上节查询的数据库结果查询行


In [4]:
p=Popen('ls -l',shell=True, stdin=PIPE, stdout=PIPE, stderr=PIPE)
# 如果调用p.communicate()会使I/O operation on closed file，即管道关闭，无法传入到下个管道
# p2的stdin选择p.stdout
p2= Popen('grep mysql', shell=True, stdin=p.stdout, stdout=PIPE, stderr=PIPE)
print(p2.stdout.read())

b'-rw-r--r-- 1 Administrator 197121  1144 Apr 14 17:50 mysql.tmp\n'


# 输出到文件

In [5]:
def mysqlPinf(user,passwd, host):
    try:
        conn = 'mysql -u%s -p%s -h%s -B' % (user,
                                            passwd, host)
        # conn = 'mysql -u%s -p%s -h%s -s'%(self.user,self.passwd,self.host)
        Pin = Popen(conn, shell=True, stdin=PIPE, stdout=open('mysql.tmp','wb'), stderr=PIPE)
        return Pin
    except Exception as e:
        print(e)
p3=mysqlPinf('root', 'paw', '127.0.0.1')
p3.stdin.write('select * from sakila.film  limit 0,5;'.encode('ascii')+b'\n')
p3.communicate()
f=open('mysql.tmp','rb')
for line in f:
    print(line.decode('utf-8'))

film_id	title	description	release_year	language_id	original_language_id	rental_duration	rental_rate	length	replacement_cost	rating	special_features	last_update

1	ACADEMY DINOSAUR	A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies	2006	1	NULL	6	0.99	86	20.99	PG	Deleted Scenes,Behind the Scenes	2006-02-15 05:03:42

2	ACE GOLDFINGER	A Astounding Epistle of a Database Administrator And a Explorer who must Find a Car in Ancient China	2006	1	NULL	3	4.99	48	12.99	G	Trailers,Deleted Scenes	2006-02-15 05:03:42

3	ADAPTATION HOLES	A Astounding Reflection of a Lumberjack And a Car who must Sink a Lumberjack in A Baloon Factory	2006	1	NULL	7	2.99	50	18.99	NC-17	Trailers,Deleted Scenes	2006-02-15 05:03:42

4	AFFAIR PREJUDICE	A Fanciful Documentary of a Frisbee And a Lumberjack who must Chase a Monkey in A Shark Tank	2006	1	NULL	5	2.99	117	26.99	G	Commentaries,Behind the Scenes	2006-02-15 05:03:42

5	AFRICAN EGG	A Fast-Paced Documentary of a Pastry Chef